In [34]:
moviesPath = "Spark/Exercises/Ex_44/movies.txt"
preferencesPath = "Spark/Exercises/Ex_44/preferences.txt"
watchedMoviesPath = "Spark/Exercises/Ex_44/watchedmovies.txt"
outputPath = "Spark/Exercises/Ex_44/res"

In [7]:
moviesRDD = sc.textFile(moviesPath)
preferencesRDD = sc.textFile(preferencesPath)
watchedMoviesRDD = sc.textFile(watchedMoviesPath)

In [8]:
watchedMoviesRDD.collect()

['user1,movie1,201606061500,201606061650',
 'user1,movie3,201606061800,201606061834',
 'user1,movie4,201609061500,201609061605',
 'user1,movie5,201610061100,201610061450',
 'user2,movie6,201610081800,201610081845',
 'user2,movie3,201610091800,201610091834',
 'user2,movie4,201611051100,201611051105']

In [4]:
preferencesRDD.collect()

['user1,Animation', 'user1,Comedy', 'user2,Action']

In [9]:
moviesRDD.collect()

['movie1,Toy Story (1995),Animation',
 'movie2,Jumanji (1995),Adventure',
 'movie3,Grumpier Old Men (1995),Comedy',
 'movie4,Waiting to Exhale (1995),Comedy',
 'movie5,Father of the Bride Part II (1995),Comedy',
 'movie6,Heat (1995),Action',
 'movie7,Sabrina (1995),Comedy',
 'movie8,Tom and Huck (1995),Adventure',
 'movie9,Sudden Death (1995),Action',
 'movie10,GoldenEye (1995),Action']

In [10]:
threshold = 60

In [11]:
movieId_genre_RDD = moviesRDD.map(lambda line : (line.split(",")[0] , line.split(",")[2]))
movieId_genre_RDD.collect()

[('movie1', 'Animation'),
 ('movie2', 'Adventure'),
 ('movie3', 'Comedy'),
 ('movie4', 'Comedy'),
 ('movie5', 'Comedy'),
 ('movie6', 'Action'),
 ('movie7', 'Comedy'),
 ('movie8', 'Adventure'),
 ('movie9', 'Action'),
 ('movie10', 'Action')]

In [15]:
user_listLikedGenres_RDD = preferencesRDD.map(lambda line : (line.split(",")[0] , line.split(",")[1])).groupByKey().mapValues(lambda listValues : list(listValues))
user_listLikedGenres_RDD.collect()

[('user1', ['Animation', 'Comedy']), ('user2', ['Action'])]

In [17]:
movie_user_RDD = watchedMoviesRDD.map(lambda line : (line.split(",")[1], line.split(",")[0]))
movie_user_RDD.collect()

[('movie1', 'user1'),
 ('movie3', 'user1'),
 ('movie4', 'user1'),
 ('movie5', 'user1'),
 ('movie6', 'user2'),
 ('movie3', 'user2'),
 ('movie4', 'user2')]

In [26]:
user_watchedGenre_RDD = movie_user_RDD.join(movieId_genre_RDD).values()
user_watchedGenre_RDD.collect()

[('user1', 'Comedy'),
 ('user1', 'Animation'),
 ('user1', 'Comedy'),
 ('user2', 'Comedy'),
 ('user1', 'Comedy'),
 ('user2', 'Comedy'),
 ('user2', 'Action')]

In [28]:
# creating an RDD with (userId, watchedGenre, likedGenres)
uId_wG_lGs_RDD = user_watchedGenre_RDD.join(user_listLikedGenres_RDD).map(lambda x : (x[0],x[1][0],x[1][1]))
uId_wG_lGs_RDD.collect()

[('user2', 'Comedy', ['Action']),
 ('user2', 'Comedy', ['Action']),
 ('user2', 'Action', ['Action']),
 ('user1', 'Comedy', ['Animation', 'Comedy']),
 ('user1', 'Animation', ['Animation', 'Comedy']),
 ('user1', 'Comedy', ['Animation', 'Comedy']),
 ('user1', 'Comedy', ['Animation', 'Comedy'])]

In [30]:
def moviesMatch(record):
    watchedGenre = record[1]
    likedGenres = record[2]
    
    if watchedGenre not in likedGenres:
        return (record[0],(1,1))
    else:
        return (record[0],(0,1))

In [33]:
finalRDD = uId_wG_lGs_RDD.map(moviesMatch).reduceByKey(lambda t1,t2 : (t1[0]+t2[0] , t1[1]+t2[1])).mapValues(lambda t : 100*t[0]/t[1]).filter(lambda t : t[1] > threshold)
finalRDD.collect()

[('user2', 66.66666666666667)]

In [36]:
finalRDD.map(lambda x : x[0]).saveAsTextFile(outputPath)